In [1]:
import numpy as np
import pandas as pd
import os
import gc
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import re
import math
import keras
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn import set_config
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import FunctionTransformer
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.layers import LayerNormalization
from keras.layers import Dropout
from keras.models import Sequential
from keras.utils import np_utils
from keras.utils.vis_utils import plot_model
from pandas.api.types import is_numeric_dtype
from itertools import product
from joblib import dump
from scipy import stats
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_selector
from sklearn.compose import make_column_transformer


In [2]:
df = pd.read_csv('/home/philippe/Documents/python project/EZStacking-main/dataset/iris.csv')

In [3]:
target_col = 'variety'

In [4]:
y = df[target_col]

In [5]:
X = df.drop(target_col, axis=1)

In [6]:
nb_features = len(X.columns.tolist())

In [7]:
nb_targets = len(y.unique())

In [8]:
layer_size = nb_features + nb_targets + 2

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [10]:
target_encoder = LabelEncoder() 
y_train = pd.Series(target_encoder.fit_transform(y_train)) 
y_test = pd.Series(target_encoder.transform(y_test))

In [11]:
num_ntree_processor = make_pipeline(SimpleImputer(strategy='mean', add_indicator=True), StandardScaler())

In [12]:
def K_Class(): 
    keras.backend.clear_session() 
#   neural network architecture: start 
    model = Sequential() 
    model.add(Dense(10 * layer_size, activation='relu')) 
    model.add(BatchNormalization()) 
    model.add(Dropout(0.5)) 
#    model.add(LayerNormalization()) 
    model.add(Dense(layer_size, activation='relu')) 
    model.add(BatchNormalization()) 
    model.add(Dropout(0.5)) 
#    model.add(LayerNormalization()) 
    model.add(Dense(nb_targets, activation='softmax')) 
#   neural network architecture: end   
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [13]:
es = EarlyStopping(monitor='val_loss', mode='auto', verbose=1, patience=20)

In [14]:
K_C = KerasClassifier(K_Class, batch_size=64, epochs=2000, callbacks=[es], validation_split=0.1, verbose=1) 
K_C._estimator_type = 'classifier'

In [15]:
level_0 = [ 
          ('LOGRL', make_pipeline(num_ntree_processor, LogisticRegression(solver='lbfgs', penalty='l2'))), 
          ('KERC', make_pipeline(num_ntree_processor, K_C)), 
          ]

In [16]:
level_1 = LogisticRegression()

In [17]:
model = StackingClassifier(level_0, final_estimator=level_1)

In [18]:
%%time 
set_config(display='diagram') 
model.fit(X_train, y_train)

Epoch 1/2000
2/2 [==============================] - 2s 360ms/step - loss: 1.4951 - accuracy: 0.3200 - val_loss: 1.1580 - val_accuracy: 0.5833
Epoch 2/2000
2/2 [==============================] - 0s 55ms/step - loss: 1.4907 - accuracy: 0.3900 - val_loss: 1.1224 - val_accuracy: 0.5833
Epoch 3/2000
2/2 [==============================] - 0s 87ms/step - loss: 1.4674 - accuracy: 0.4200 - val_loss: 1.0866 - val_accuracy: 0.5833
Epoch 4/2000
2/2 [==============================] - 0s 57ms/step - loss: 1.3102 - accuracy: 0.4200 - val_loss: 1.0549 - val_accuracy: 0.6667
Epoch 5/2000
2/2 [==============================] - 0s 68ms/step - loss: 1.5313 - accuracy: 0.4100 - val_loss: 1.0241 - val_accuracy: 0.7500
Epoch 6/2000
2/2 [==============================] - 0s 74ms/step - loss: 1.4254 - accuracy: 0.3400 - val_loss: 0.9978 - val_accuracy: 0.7500
Epoch 7/2000
2/2 [==============================] - 0s 64ms/step - loss: 1.2566 - accuracy: 0.4600 - val_loss: 0.9730 - val_accuracy: 0.7500
Epoch 8/2000

Epoch 59/2000
2/2 [==============================] - 0s 77ms/step - loss: 0.4565 - accuracy: 0.7900 - val_loss: 0.5528 - val_accuracy: 0.7500
Epoch 60/2000
2/2 [==============================] - 0s 64ms/step - loss: 0.4696 - accuracy: 0.7900 - val_loss: 0.5487 - val_accuracy: 0.7500
Epoch 61/2000
2/2 [==============================] - 0s 64ms/step - loss: 0.3894 - accuracy: 0.8700 - val_loss: 0.5439 - val_accuracy: 0.7500
Epoch 62/2000
2/2 [==============================] - 0s 64ms/step - loss: 0.4134 - accuracy: 0.8200 - val_loss: 0.5394 - val_accuracy: 0.7500
Epoch 63/2000
2/2 [==============================] - 0s 66ms/step - loss: 0.4531 - accuracy: 0.8500 - val_loss: 0.5349 - val_accuracy: 0.7500
Epoch 64/2000
2/2 [==============================] - 0s 60ms/step - loss: 0.4112 - accuracy: 0.8300 - val_loss: 0.5304 - val_accuracy: 0.7500
Epoch 65/2000
2/2 [==============================] - 0s 56ms/step - loss: 0.4031 - accuracy: 0.8800 - val_loss: 0.5262 - val_accuracy: 0.7500
Epoch 

2/2 [==============================] - 0s 59ms/step - loss: 0.2972 - accuracy: 0.9100 - val_loss: 0.3362 - val_accuracy: 0.9167
Epoch 117/2000
2/2 [==============================] - 0s 62ms/step - loss: 0.2943 - accuracy: 0.9100 - val_loss: 0.3333 - val_accuracy: 0.9167
Epoch 118/2000
2/2 [==============================] - 0s 67ms/step - loss: 0.3673 - accuracy: 0.8200 - val_loss: 0.3306 - val_accuracy: 0.9167
Epoch 119/2000
2/2 [==============================] - 0s 73ms/step - loss: 0.3130 - accuracy: 0.8700 - val_loss: 0.3277 - val_accuracy: 0.9167
Epoch 120/2000
2/2 [==============================] - 0s 64ms/step - loss: 0.3389 - accuracy: 0.9100 - val_loss: 0.3247 - val_accuracy: 0.9167
Epoch 121/2000
2/2 [==============================] - 0s 63ms/step - loss: 0.2788 - accuracy: 0.9000 - val_loss: 0.3220 - val_accuracy: 0.9167
Epoch 122/2000
2/2 [==============================] - 0s 63ms/step - loss: 0.3066 - accuracy: 0.8900 - val_loss: 0.3198 - val_accuracy: 0.9167
Epoch 123/2000

2/2 [==============================] - 0s 64ms/step - loss: 0.2299 - accuracy: 0.9300 - val_loss: 0.2279 - val_accuracy: 0.9167
Epoch 174/2000
2/2 [==============================] - 0s 58ms/step - loss: 0.2032 - accuracy: 0.9200 - val_loss: 0.2269 - val_accuracy: 0.9167
Epoch 175/2000
2/2 [==============================] - 0s 70ms/step - loss: 0.2221 - accuracy: 0.9500 - val_loss: 0.2259 - val_accuracy: 0.9167
Epoch 176/2000
2/2 [==============================] - 0s 63ms/step - loss: 0.2338 - accuracy: 0.9200 - val_loss: 0.2248 - val_accuracy: 0.8333
Epoch 177/2000
2/2 [==============================] - 0s 65ms/step - loss: 0.2133 - accuracy: 0.9000 - val_loss: 0.2241 - val_accuracy: 0.8333
Epoch 178/2000
2/2 [==============================] - 0s 57ms/step - loss: 0.2457 - accuracy: 0.9100 - val_loss: 0.2237 - val_accuracy: 0.8333
Epoch 179/2000
2/2 [==============================] - 0s 51ms/step - loss: 0.2632 - accuracy: 0.9100 - val_loss: 0.2234 - val_accuracy: 0.8333
Epoch 180/2000

2/2 [==============================] - 0s 72ms/step - loss: 0.2054 - accuracy: 0.9000 - val_loss: 0.2097 - val_accuracy: 0.9167
Epoch 231/2000
2/2 [==============================] - 0s 63ms/step - loss: 0.1958 - accuracy: 0.9300 - val_loss: 0.2102 - val_accuracy: 0.9167
Epoch 232/2000
2/2 [==============================] - 0s 61ms/step - loss: 0.2230 - accuracy: 0.9100 - val_loss: 0.2104 - val_accuracy: 0.9167
Epoch 233/2000
2/2 [==============================] - 0s 53ms/step - loss: 0.2451 - accuracy: 0.8800 - val_loss: 0.2106 - val_accuracy: 0.9167
Epoch 234/2000
2/2 [==============================] - 0s 74ms/step - loss: 0.1324 - accuracy: 0.9700 - val_loss: 0.2116 - val_accuracy: 0.9167
Epoch 235/2000
2/2 [==============================] - 0s 57ms/step - loss: 0.1934 - accuracy: 0.9400 - val_loss: 0.2125 - val_accuracy: 0.9167
Epoch 236/2000
2/2 [==============================] - 0s 68ms/step - loss: 0.1825 - accuracy: 0.9400 - val_loss: 0.2143 - val_accuracy: 0.9167
Epoch 237/2000

...

2/2 [==============================] - 0s 69ms/step - loss: 0.2845 - accuracy: 0.9012 - val_loss: 0.1770 - val_accuracy: 1.0000
Epoch 156/2000
2/2 [==============================] - 0s 59ms/step - loss: 0.3684 - accuracy: 0.8395 - val_loss: 0.1746 - val_accuracy: 1.0000
Epoch 157/2000
2/2 [==============================] - 0s 64ms/step - loss: 0.3164 - accuracy: 0.9012 - val_loss: 0.1730 - val_accuracy: 1.0000
Epoch 158/2000
2/2 [==============================] - 0s 70ms/step - loss: 0.2875 - accuracy: 0.9136 - val_loss: 0.1709 - val_accuracy: 1.0000
Epoch 159/2000
2/2 [==============================] - 0s 51ms/step - loss: 0.2239 - accuracy: 0.9630 - val_loss: 0.1687 - val_accuracy: 1.0000
Epoch 160/2000
2/2 [==============================] - 0s 63ms/step - loss: 0.2677 - accuracy: 0.8889 - val_loss: 0.1666 - val_accuracy: 1.0000
Epoch 161/2000
2/2 [==============================] - 0s 66ms/step - loss: 0.2801 - accuracy: 0.9136 - val_loss: 0.1644 - val_accuracy: 1.0000
Epoch 162/2000

2/2 [==============================] - 0s 77ms/step - loss: 0.2135 - accuracy: 0.9259 - val_loss: 0.0918 - val_accuracy: 1.0000
Epoch 213/2000
2/2 [==============================] - 0s 60ms/step - loss: 0.2891 - accuracy: 0.8765 - val_loss: 0.0904 - val_accuracy: 1.0000
Epoch 214/2000
2/2 [==============================] - 0s 57ms/step - loss: 0.2331 - accuracy: 0.9506 - val_loss: 0.0888 - val_accuracy: 1.0000
Epoch 215/2000
2/2 [==============================] - 0s 71ms/step - loss: 0.2744 - accuracy: 0.9012 - val_loss: 0.0878 - val_accuracy: 1.0000
Epoch 216/2000
2/2 [==============================] - 0s 61ms/step - loss: 0.1845 - accuracy: 0.9753 - val_loss: 0.0865 - val_accuracy: 1.0000
Epoch 217/2000
2/2 [==============================] - 0s 61ms/step - loss: 0.2207 - accuracy: 0.9012 - val_loss: 0.0853 - val_accuracy: 1.0000
Epoch 218/2000
2/2 [==============================] - 0s 65ms/step - loss: 0.2188 - accuracy: 0.9136 - val_loss: 0.0840 - val_accuracy: 1.0000
Epoch 219/2000

2/2 [==============================] - 0s 64ms/step - loss: 0.1686 - accuracy: 0.9753 - val_loss: 0.0450 - val_accuracy: 1.0000
Epoch 270/2000
2/2 [==============================] - 0s 59ms/step - loss: 0.2216 - accuracy: 0.9012 - val_loss: 0.0444 - val_accuracy: 1.0000
Epoch 271/2000
2/2 [==============================] - 0s 65ms/step - loss: 0.1831 - accuracy: 0.9506 - val_loss: 0.0436 - val_accuracy: 1.0000
Epoch 272/2000
2/2 [==============================] - 0s 63ms/step - loss: 0.2077 - accuracy: 0.9383 - val_loss: 0.0434 - val_accuracy: 1.0000
Epoch 273/2000
2/2 [==============================] - 0s 68ms/step - loss: 0.1814 - accuracy: 0.9383 - val_loss: 0.0428 - val_accuracy: 1.0000
Epoch 274/2000
2/2 [==============================] - 0s 73ms/step - loss: 0.1735 - accuracy: 0.9506 - val_loss: 0.0423 - val_accuracy: 1.0000
Epoch 275/2000
2/2 [==============================] - 0s 63ms/step - loss: 0.2217 - accuracy: 0.9259 - val_loss: 0.0424 - val_accuracy: 1.0000
Epoch 276/2000

2/2 [==============================] - 0s 55ms/step - loss: 0.1705 - accuracy: 0.9383 - val_loss: 0.0255 - val_accuracy: 1.0000
Epoch 327/2000
2/2 [==============================] - 0s 61ms/step - loss: 0.1101 - accuracy: 0.9877 - val_loss: 0.0260 - val_accuracy: 1.0000
Epoch 328/2000
2/2 [==============================] - 0s 66ms/step - loss: 0.2038 - accuracy: 0.9259 - val_loss: 0.0264 - val_accuracy: 1.0000
Epoch 329/2000
2/2 [==============================] - 0s 57ms/step - loss: 0.2027 - accuracy: 0.9012 - val_loss: 0.0267 - val_accuracy: 1.0000
Epoch 330/2000
2/2 [==============================] - 0s 65ms/step - loss: 0.1863 - accuracy: 0.9506 - val_loss: 0.0268 - val_accuracy: 1.0000
Epoch 331/2000
2/2 [==============================] - 0s 62ms/step - loss: 0.1843 - accuracy: 0.9259 - val_loss: 0.0268 - val_accuracy: 1.0000
Epoch 332/2000
2/2 [==============================] - 0s 76ms/step - loss: 0.1613 - accuracy: 0.9383 - val_loss: 0.0268 - val_accuracy: 1.0000
Epoch 333/2000

StackingClassifier(estimators=[('LOGRL',
                                Pipeline(steps=[('pipeline',
                                                 Pipeline(steps=[('simpleimputer',
                                                                  SimpleImputer(add_indicator=True)),
                                                                 ('standardscaler',
                                                                  StandardScaler())])),
                                                ('logisticregression',
                                                 LogisticRegression())])),
                               ('KERC',
                                Pipeline(steps=[('pipeline',
                                                 Pipeline(steps=[('simpleimputer',
                                                                  SimpleImputer(add_indicator=True)),
                                                                 ('standardscaler',
                                                                  StandardScaler())])),
                                                ('kerasclassifier',
                                                 <keras.wrappers.scikit_learn.KerasClassifier object at 0x7f6c2c6f0d60>)]))],
                   final_estimator=LogisticRegression())

In [19]:
model.score(X_test, y_test)

2/2 [==============================] - 0s 5ms/step


1.0